In [2]:
import pandas as pd
import numpy as np
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows as df_to_row
import pathlib

In [3]:
from szp_funcs import get_fio, custom_create_res, print_df, load_groups, gku_inn, path_to_docs, to_double, path_to_data, load_month, gku, load_month_all

In [4]:
month = 'jun'
ped, isp, aup = load_groups('input')
out_dir = str(pathlib.Path().resolve()) + '\\200+_' + month + '\\'

In [26]:
inns = [(7717043346, 'ГАОУ ВО МГПУ'), (7723356160, 'ГБПОУ 26 КАДР'), (7734570529, 'ГАУДО Домисолька'), (7725247657, 'ГБПОУ ОКГ Столица'), (7733020995,'ГБПОУ Колледж полиции'), (7704040242, 'ГБОУ Школа № 57'), (7736110982, 'ГБПОУ Воробьевы горы'), (7731281039, 'ГБПОУ МИК'), (7731174051, 'ГБОУ Школа № 1371 Крылатское')]

In [6]:
ppl = pd.read_excel(path_to_docs + 'school_ppl_' + month + '.xlsx')
ppl = ppl[ppl.ppl > 0]

In [8]:
fail = pd.read_excel(path_to_docs + '200_ppl_' + month + '.xlsx')['snils'].to_list()

In [9]:
fails = pd.read_excel(path_to_docs + month + '_inn_fail.xlsx')['inn'].to_list()

In [10]:
jobs = pd.read_excel(path_to_docs + '200_jobs_' + month + '.xlsx')

In [11]:
res = load_month(month)

In [12]:
res = res[(res['sum_' + month] >= 54500) & (res.inn.isin(gku_inn) == False)]

In [13]:
def make_grp(row):
    if row['ppl'] > 4000:
        return 1
    if row['ppl'] < 2000:
        return 3
    return 2

In [14]:
for_tables = pd.merge(pd.merge(res[['inn', 'job_' + month, 'sum_' + month, 'snils']], jobs, how='left', left_on='job_' + month, right_on='job').drop('job', axis=1), ppl, how='left')
for_tables['grp'] = for_tables.apply(lambda row: make_grp(row), axis=1)

In [15]:
for_tables

,inn,job_jun,sum_jun,snils,job_grp,ppl,grp
0,5003021368,Помощник воспитателя,62730.03,001-171-362 75,NaN,10722.0,1
1,5003021368,Учитель,121248.22,001-244-107 65,NaN,10722.0,1
2,5003021368,Специалист,75815.00,001-322-594 80,Специалисты,10722.0,1
3,5003021368,Учитель,182893.03,001-519-663 10,NaN,10722.0,1
4,5003021368,Воспитатель,63929.02,001-752-213 93,NaN,10722.0,1
...,...,...,...,...,...,...,...
133359,9715217689,Учитель,124031.31,187-933-260 19,NaN,2488.0,2
133360,9715217689,Учитель,160046.33,191-336-708 73,NaN,2488.0,2
133361,9715217689,Педагог-организатор,115325.80,195-450-909 00,NaN,2488.0,2
133362,9715217689,Учитель,97717.75,196-998-174 73,NaN,2488.0,2


In [16]:
szp_all = for_tables[['job_grp', 'sum_' + month]].groupby(['job_grp']).mean().reset_index()
szp_all_min = for_tables[['job_grp', 'sum_' + month]].groupby(['job_grp']).min().reset_index()
szp_all_max = for_tables[['job_grp', 'sum_' + month]].groupby(['job_grp']).max().reset_index()

In [17]:
szp_all = pd.merge(szp_all, szp_all_min, how='left', on=['job_grp'])
szp_all = pd.merge(szp_all, szp_all_max, how='left', on=['job_grp'])

In [18]:
szp_all = szp_all.rename(columns={'job_'  + month:'job', 'sum_'  + month:'max', 'sum_' + month +'_x':'mean', 'sum_' + month +'_y':'min'})

In [19]:
szp_all

,job_grp,mean,min,max
0,IT - специалисты,125749.413387,54500.00,336002.32
1,Администраторы и секретари,92837.084839,54500.00,314566.69
2,Ведущие экономисты (экономисты),136083.176628,54549.97,443607.81
3,Контрактные управляющие и специалисты по закупкам,119875.451120,55304.00,363278.96
4,Руководящий состав среднего звена,123146.924017,54500.00,563642.67
5,Специалисты,103171.619796,54500.00,756568.17
6,Специалисты по кадрам и юристконсульты,105933.901351,54500.00,380102.00
7,Специалисты по обслуживанию и содержанию здани...,108756.735286,54500.00,409395.06


In [20]:
print_df(szp_all, out_dir + 'СЗП + вилка все по группам ' + month)

In [21]:
szp = for_tables[['grp', 'job_grp', 'sum_' + month]].groupby(['grp', 'job_grp']).mean().reset_index()
szp_max = for_tables[['grp', 'job_grp', 'sum_' + month]].groupby(['grp', 'job_grp']).max().reset_index()
szp_min = for_tables[['grp', 'job_grp', 'sum_' + month]].groupby(['grp', 'job_grp']).min().reset_index()
szp = pd.merge(szp, szp_min, how='left', on=['grp', 'job_grp'])
szp = pd.merge(szp, szp_max, how='left', on=['grp', 'job_grp'])
szp = szp.rename(columns={'job_'  + month:'job', 'sum_'  + month:'max', 'sum_' + month +'_x':'mean', 'sum_' + month +'_y':'min'})


In [22]:
min_good_sch = for_tables[(for_tables.inn.isin(fails) == False) & (for_tables['sum_' + month] < 200000)][['grp', 'job_grp', 'sum_' + month]].groupby(['grp', 'job_grp']).min().reset_index().rename(columns={'sum_' + month: 'min_good_sch'})
max_good_sch = for_tables[(for_tables.inn.isin(fails) == False) & (for_tables['sum_' + month] < 200000)][['grp', 'job_grp', 'sum_' + month]].groupby(['grp', 'job_grp']).max().reset_index().rename(columns={'sum_' + month: 'max_good_sch'})
szp_good_sch = for_tables[(for_tables.inn.isin(fails) == False) & (for_tables['sum_' + month] < 200000)][['grp', 'job_grp', 'sum_' + month]].groupby(['grp', 'job_grp']).mean().reset_index().rename(columns={'sum_' + month: 'szp_good_sch'})

min_bad_sch_bad_ppl = for_tables[(for_tables.inn.isin(fails)) & (for_tables['sum_' + month] > 200000)][['grp', 'job_grp', 'sum_' + month]].groupby(['grp', 'job_grp']).min().reset_index().rename(columns={'sum_' + month: 'min_bad_sch_bad_ppl'})
max_bad_sch_bad_ppl = for_tables[(for_tables.inn.isin(fails)) & (for_tables['sum_' + month] > 200000)][['grp', 'job_grp', 'sum_' + month]].groupby(['grp', 'job_grp']).max().reset_index().rename(columns={'sum_' + month: 'max_bad_sch_bad_ppl'})
szp_bad_sch_bad_ppl = for_tables[(for_tables.inn.isin(fails)) & (for_tables['sum_' + month] > 200000)][['grp', 'job_grp', 'sum_' + month]].groupby(['grp', 'job_grp']).mean().reset_index().rename(columns={'sum_' + month: 'szp_bad_sch_bad_ppl'})

min_bad_sch_good_ppl = for_tables[(for_tables.inn.isin(fails)) & (for_tables['sum_' + month] < 200000)][['grp', 'job_grp', 'sum_' + month]].groupby(['grp', 'job_grp']).min().reset_index().rename(columns={'sum_' + month: 'min_bad_sch_good_ppl'})
max_bad_sch_good_ppl = for_tables[(for_tables.inn.isin(fails)) & (for_tables['sum_' + month] < 200000)][['grp', 'job_grp', 'sum_' + month]].groupby(['grp', 'job_grp']).max().reset_index().rename(columns={'sum_' + month: 'max_bad_sch_good_ppl'})
szp_bad_sch_good_ppl = for_tables[(for_tables.inn.isin(fails)) & (for_tables['sum_' + month] < 200000)][['grp', 'job_grp', 'sum_' + month]].groupby(['grp', 'job_grp']).mean().reset_index().rename(columns={'sum_' + month: 'szp_bad_sch_good_ppl'})

In [23]:
szp = pd.merge(szp, min_good_sch, how='left', on=['grp', 'job_grp'])
szp = pd.merge(szp, max_good_sch, how='left', on=['grp', 'job_grp'])
szp = pd.merge(szp, szp_good_sch, how='left', on=['grp', 'job_grp'])
szp = pd.merge(szp, min_bad_sch_bad_ppl, how='left', on=['grp', 'job_grp'])
szp = pd.merge(szp, max_bad_sch_bad_ppl, how='left', on=['grp', 'job_grp'])
szp = pd.merge(szp, szp_bad_sch_bad_ppl, how='left', on=['grp', 'job_grp'])
szp = pd.merge(szp, min_bad_sch_good_ppl, how='left', on=['grp', 'job_grp'])
szp = pd.merge(szp, max_bad_sch_good_ppl, how='left', on=['grp', 'job_grp'])
szp = pd.merge(szp, szp_bad_sch_good_ppl, how='left', on=['grp', 'job_grp'])

In [24]:
print_df(szp, out_dir + 'СЗП + вилка все по контингенту ' + month)

In [27]:
for sch in inns:
    szp_t = for_tables[(for_tables['inn'] == sch[0]) & (for_tables['sum_' + month] > 200000) & (for_tables['snils'].isin(fail))][['job_grp', 'sum_' + month]].groupby(['job_grp']).mean().reset_index()
    szp_t_min = for_tables[(for_tables['inn'] == sch[0]) & (for_tables['sum_' + month] > 200000) & (for_tables['snils'].isin(fail))][['job_grp', 'sum_' + month]].groupby(['job_grp']).min().reset_index()
    szp_t_max = for_tables[(for_tables['inn'] == sch[0]) & (for_tables['sum_' + month] > 200000) & (for_tables['snils'].isin(fail))][['job_grp', 'sum_' + month]].groupby(['job_grp']).max().reset_index()
    szp_t_cnt = for_tables[(for_tables['inn'] == sch[0]) & (for_tables['sum_' + month] > 200000) & (for_tables['snils'].isin(fail))][['job_grp', 'sum_' + month]].groupby(['job_grp']).count().reset_index().rename(columns={'sum_' + month: 'cnt'})
    szp_t = pd.merge(szp_t, szp_t_min, how='left', on=['job_grp'])
    szp_t = pd.merge(szp_t, szp_t_max, how='left', on=['job_grp'])
    szp_t = pd.merge(szp_t, szp_t_cnt, how='left', on=['job_grp'])
    szp_t = szp_t.rename(columns={'job_'  + month:'job', 'sum_'  + month:'max', 'sum_' + month +'_x':'mean', 'sum_' + month +'_y':'min'})
    print_df(szp_t, out_dir + 'Вилка организации 200+ ' + month + ' ' + sch[1])